# A demo how to work with QAOA Parameter classes

This notebook illustrates the use of the various QAOA parameter classes.

1. [Creating the problem Hamiltonian and setting up *hyperparameters*](#hyperparameters)
2. [QAOA variable parameters](#variableparams)
 - [Setting up the circuit parameters](#abstractparams)
 - [Modifying parameters](#modifyingparams)
 - [Erroneous parameter catching](#errorcatching) 
3. [Iterating over parameter ranges](#iteratingparams)

In [1]:
# import the standard modules from python
import numpy as np
import matplotlib.pyplot as plt
from typing import Type, Iterable

# import the neccesary pyquil modules
from pyquil.paulis import PauliSum, PauliTerm

# import the QAOAParameters classes 
from entropica_qaoa.qaoa.parameters import AbstractParams, ExtendedParams, StandardParams, AnnealingParams,\
                                        FourierParams, QAOAParameterIterator

<a id="hyperparameters"></a>


## Creating the problem Hamiltonian and setting up *hyperparameters*

In general, the QAOA consists of two different types of parameters, which we will refer to as *hyperparameters* and *variable parameters*. This section covers the hyperparameters, while the section below focuses on the variable parameters. The hyperparameters are those parameters that remain fixed throughout our computation, while the variable parameters are those that we modify in seeking the optimial problem solution.

In the simplest implementation of QAOA, the hyperparameters may in turn be divided into two sets (see [Footnote 1](#footnotes) for a third example of hyperparameters):

1) Those originating from the cost Hamiltonian: 

  - the qubit register (the qubits to be used in the algorithm); 
  - the qubits with a bias term (their own $Z$ term in the Hamiltonian), and the corresponding coefficients; 
  - the qubit pairs that interact (through a $ZZ$ term in the Hamiltonian), along with the corresponding 'coupling' coefficients.

2) The number of QAOA steps we wish to perform, frequently referred to as the QAOA '$p$' parameter.

There are several ways of creating the problem Hamiltonian of interest. Ultimately, it should be in the form of a Pyquil `PauliSum` object: see __[Rigetti's documentation](http://docs.rigetti.com/en/stable/apidocs/pauli.html#)__ on the `PauliSum` and `PauliTerm` classes for more information on working with these. 

For example, let's create the simple Hamiltonian
 
\begin{equation}
H = 0.7Z_0Z_1 + 1.2Z_0Z_2 -0.5Z_0
\end{equation}

Using `PauliTerm` and `PauliSum` directly, this can be implemented as follows.

In [2]:
# create a hamiltonian on 3 qubits with 2 coupling terms and 1 bias term
Term1 = PauliTerm("Z", 0, 0.7)*PauliTerm("Z", 1)
Term2 = PauliTerm("Z", 0, 1.2)*PauliTerm("Z", 2)
Term3 = PauliTerm("Z", 0, -0.5)

hamiltonian = PauliSum([Term1,Term2,Term3])
print("hamiltonian =", hamiltonian)

hamiltonian = (0.7+0j)*Z0*Z1 + (1.2+0j)*Z0*Z2 + (-0.5+0j)*Z0


<a id="variableparams"></a>

## QAOA variable parameter classes

Having specified the problem Hamiltonian, we next move on to defining the QAOA parameters we want to use in our quantum circuit.

For a general variational quantum algorithm usch as VQE, to fully define a problem we must specify a circuit ansatz (a sequence of gates to be performed) and a corresponding parametrisation (how the parameters over which we intend to optimise are related to the sequence of gates). In QAOA, the circuit ansatz is fixed to be the alternate application of the mixer operator (also referred to as the *driver* or *reference* operator) and the cost operator (sometimes also referred to as the *phase separation* operator). The goal is then to find the parameters that optimise the cost function when evaluated with resepct to the quantum state produced by the circuit. 

We have a considerable degree of flexibility in choosing how to parametrise a QAOA problem. We can choose a smaller set of parameters, where the optimisation landscape has lower dimension at the expense of reduced expressivity. Or, we can choose a larger set, where we can generate a wider set of quantum states with lower circuit depth, but the corresponding optimisation landscape has a higher dimension. The larger the set of parameters we have to optimise over, the more difficult it can become to find the optimal solution.


The variable parameters are those we wish to optimise, which in turn depend on the specific parametrisation for the circuit we have chosen. 

- The `StandardParams` class implements the original form of the QAOA, as described in [Ref 1](#references). In time step $q$ of the algorithm, the mixer and cost Hamiltonians are applied with coefficients $\beta^{(q)}$ and $\gamma^{(q)}$, respectively, giving a total of $2p$ parameters over which we need to optimise. For example for a depth-2 ($p=2$) circuit, the unitary operator corresponding to the QAOA circuit would take the form

\begin{equation}
U(\beta_1,\beta_2,\gamma_1,\gamma_2) = \exp\left(i\beta^{(2)} H_{M}\right)\exp\left(i\gamma^{(2)} H_{C}\right)\exp\left(i\beta^{(1)} H_{M}\right)\exp\left(i\gamma^{(1)} H_{C}\right)
\end{equation}

where the mixer Hamiltonian is given by $H_M = \sum_j X_j$, and the cost Hamiltonian is given by $H_C = \sum_j h_j Z_j + (1/2)\sum_{j,k} g_{j,k} Z_jZ_k$.

- For the `ExtendedParams` class, each operator in both the cost and mixer Hamiltonians has its own angle, so that the set of variable parameters are:

    - $\mbox{betas} = \{\beta_0^{(1)},...,\beta_{n-1}^{(1)},\beta_0^{(2)},...,\beta_{n-1}^{(2)},...,\beta_0^{(p)},...,\beta_{n-1}^{(p)}\}$, where $\beta_i^{(q)}$ denotes the mixer Hamiltonian angle for qubit $i$ in the QAOA step $q$.
    - $\mbox{gammas_singles} = \left\{ \{\gamma_s^{(1)}\}, \{\gamma_s^{(2)}\},...,\{\gamma_s^{(p)}\}    \right\}$, where  where $s$ is the set of qubits with bias terms in the cost Hamiltonian, and $\{\gamma_s^{(q)}\}$ denotes the set of angles corresponding to those bias terms in QAOA step $q$.  
    - $\mbox{gammas_pairs} = \left\{ \{\Gamma_{\Pi}^{(1)}\}, \{\Gamma_{\Pi}^{(2)}\},...,\{\Gamma_{\Pi}^{(p)}\}    \right\}$, where  where $\Pi$ is the set of qubits with bias terms in the cost Hamiltonian, and $\{\Gamma_{\Pi}^{(q)}\}$ denotes the set of angles corresponding to those bias terms in QAOA step $q$.  
    
    For instance, for a depth-2 circuit the corresponding unitary operator would then become:
    
\begin{eqnarray}
U\left(\vec{\beta},\vec{\gamma},\vec{\Gamma}\right) = &&\exp\left(i\sum_{j}\beta_{j}^{(2)}X_j\right)\exp\left(i\sum_{j\in s} \gamma_{j}^{(2)}h_{j}Z_j + (i/2)\sum_{j,k \in \Pi}\Gamma_{jk}^{(2)}g_{jk}Z_jZ_k\right) \\
&\times&\exp\left(i\sum_{j}\beta_{j}^{(1)}X_j\right)\exp\left(i\sum_{j\in s} \gamma_{j}^{(1)}h_{j}Z_j + (i/2)\sum_{j,k \in \Pi}\Gamma_{jk}^{(1)}g_{jk}Z_jZ_k\right)
\end{eqnarray}
    
We currently provide two additional parameter classes that may be of interest, either for didactic or practical purposes.
    
- `AdiabaticParams` [Explanation and demo to be completed] - basically a digitised form of annealing, where the coefficient of the mixer Hamiltonian is a function s(t), and the coefficient of the cost Hamiltonian is just (1 - s(t)), so unlike QAOA, the coefficients of the two Hamiltonians are necessarily related to one another.

- `FourierParams` [Explanation and demo to be completed] - see [Ref 2](#references).

<a id="abstractparams"></a>

### Building parameters from the `AbstractParams` class

All of the QAOA parameter classes listed above are descendants of the parent class `AbstractParams`. An object belonging to this class is characterised by the problem hyperparameters. We create such an object as follows:

In [3]:
timesteps = 2
abstract_params = AbstractParams([hamiltonian,timesteps])
print(abstract_params)

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	single_qubit_coeffs: [-0.5]
	qubits_pairs: [[0, 1], [0, 2]]
	pair_qubit_coeffs: [0.7 1.2]
	n_steps: 2



Subsequently, we can initalise any of the sets of variable parameters described above by making use of the `AbstractParams` object. For example, let's set up `ExtendedParams` using the `AbstractParams` we have just defined. To do this, we call the `.from_AbstractParameters` method.

In [4]:
# Specify some angles
betas          = [[0.0, 0.1, 0.3], [0.5, 0.2, 1.2]]
gammas_singles = [[0.0], [0.5]]
gammas_pairs   = [[0.1, 0.3], [0.2, 1.2]]
parameters = (betas, gammas_singles, gammas_pairs)

extended_params = ExtendedParams.from_AbstractParameters(abstract_params,parameters)
print(extended_params)

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [[0.  0.1 0.3], [0.5 0.2 1.2]]
	gammas_singles: [[0. ], [0.5]]
	gammas_pairs: [[0.1 0.3], [0.2 1.2]]



Note that we can also create `ExtendedParams` directly, without first setting up `AbstractParams`. For the case considered above, we would simply do the following:

In [5]:
extended_direct = ExtendedParams([hamiltonian,2],parameters)
print(extended_direct)

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [[0.  0.1 0.3], [0.5 0.2 1.2]]
	gammas_singles: [[0. ], [0.5]]
	gammas_pairs: [[0.1 0.3], [0.2 1.2]]



We could set up an object of any of the other parameter classes analogously to the case we have illustrated above for `ExtendedParams`, either directly, or through `AbstractParams`. The benefit of the latter method is that it allows multiple different parametrisations to be set up from the same underlying set of `hyperparameters`.


For all parameter classes, we also provide the `.linear_ramp_from_hamiltonian()` method, which automatically produces a set of parameters obeying a linear ramp for the number of timesteps specified. [TODO: EXPLAIN THIS IN MORE DETAIL HERE OR ELSEWHERE]. For example:

In [6]:
timesteps = 2      # equivalent to p in Farhi's original QAOA paper
time = 1            # total time of the annealing schedule
params = StandardParams.linear_ramp_from_hamiltonian(hamiltonian, timesteps, time = time) 
print(params)

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [0.375 0.125]
	gammas: [0.125 0.375]



We see that if we print `params`, it gives us the timesteps, the angles, and on which combinations of qubits they act. But we can also have a look at the _internal_ `x_rotation_angles`, `z_rotation_angles` and `zz_rotation_angles` that were automatically calculated under the hood. These are the rotation angles for the different Pauli operators in the actual execution of the quantum circuit, i.e. in the time evolution operator (the exponentiated Hamiltonian); they are determined by products of the `parameters` and the `coefficients` in the Hamiltonian itself [TODO: REPLACE `coefficients` WITH THE ACTUAL NAMES OF THE PARAMS. ALSO WORD THIS BETTER TO MAKE CLEAR WHAT HAPPENS WITH THE MIXER HAMILTONIAN].

In [7]:
print("\n x_rotation_angles:\n", params.x_rotation_angles)
print("\n z_rotation_angles:\n", params.z_rotation_angles)
print("\n zz_rotation_angles:\n", params.zz_rotation_angles)


 x_rotation_angles:
 [[0.375 0.375 0.375]
 [0.125 0.125 0.125]]

 z_rotation_angles:
 [[-0.0625]
 [-0.1875]]

 zz_rotation_angles:
 [[0.0875 0.15  ]
 [0.2625 0.45  ]]


In a standard workflow, we would then pass these `params` to a cost_function that has been built with `qaoa.cost_function.QAOACostFunctionOnQVM` or with `qaoa.cost_function.QAOACostFunctionOnWFSim`. To do so, we first arrange the `params` in a single 'raw' list of parameters in the form [`betas`,`gammas_singles`,`gammas_pairs`]:

In [8]:
raw_params = params.raw()
raw_params

array([0.375, 0.125, 0.125, 0.375])

<a id="modifyingparams"></a>

## Modifying parameters

We may wish to modify the parameters that have been set up, before running QAOA with them. The two ways we can do this are to modify the `params` or the `raw_params`.

In [9]:
# Current params and under-the-hood equivalents
print(params)
print("\n x_rotation_angles:\n", params.x_rotation_angles)
print("\n z_rotation_angles:\n", params.z_rotation_angles)
print("\n zz_rotation_angles:\n", params.zz_rotation_angles)

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [0.375 0.125]
	gammas: [0.125 0.375]


 x_rotation_angles:
 [[0.375 0.375 0.375]
 [0.125 0.125 0.125]]

 z_rotation_angles:
 [[-0.0625]
 [-0.1875]]

 zz_rotation_angles:
 [[0.0875 0.15  ]
 [0.2625 0.45  ]]


Let's modify the first `betas` parameter:

In [10]:
params.betas[0] = np.pi
print(params)
print("\n x_rotation_angles:\n", params.x_rotation_angles)
print("\n z_rotation_angles:\n", params.z_rotation_angles)
print("\n zz_rotation_angles:\n", params.zz_rotation_angles)

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [3.14159265 0.125     ]
	gammas: [0.125 0.375]


 x_rotation_angles:
 [[3.14159265 3.14159265 3.14159265]
 [0.125      0.125      0.125     ]]

 z_rotation_angles:
 [[-0.0625]
 [-0.1875]]

 zz_rotation_angles:
 [[0.0875 0.15  ]
 [0.2625 0.45  ]]


If we know the index of the parameter(s) we wish to vary in the `raw` list, we can also modify them there.

<a id="errorcatching"></a>

### What happens when an invalid set of parameters are passed to a `QAOAParameter` class?

Here give an example of what happens if the user specifies too many or too few parameters for the choice of parametrisation they want to use...

<a id="iteratingparams"></a>

## Iterating over parameter ranges

We may sometimes want to investigate what happens if we vary one of the parameters over some specfied range, keeping all others fixed. The `QAOAParameterIterator` class gives a convenient way of doing this.

For the parameters above, we could for example take one of the `gammas` to iterate over. Suppose we take the second one, `gammas[1]`, and we want to vary it in steps of size 1/3 between 0 and 1:

In [11]:
the_range = np.arange(0, 1, 0.3334)
the_parameter = "gammas[1]"
iterator= QAOAParameterIterator(params, the_parameter, the_range)
for i,p in zip(range(len(the_range)),iterator):
    print('Parameters at step' + str(i) + ':')
    print(p)

Parameters at step0:
Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [3.14159265 0.125     ]
	gammas: [0.125 0.   ]

Parameters at step1:
Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [3.14159265 0.125     ]
	gammas: [0.125  0.3334]

Parameters at step2:
Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [3.14159265 0.125     ]
	gammas: [0.125  0.6668]



As expected, in the loop all parameters stay constant except for `gammas[1]` which went up in steps of thirds. 

Of course, we can also nest these iterators to sweep two-dimensional paramter landscapes. For instance, suppose we now sweep over parameter ranges for both `gammas[1]` and `betas[0]`:

In [12]:
range1 = np.arange(0,1,0.5)
range2 = np.arange(2,3,0.5)
iterator1 = QAOAParameterIterator(params, "gammas[1]", range1)
for j in iterator1:
    for p in QAOAParameterIterator(j, "betas[0]", range2):
        print("betas =", p.betas, ",", "gammas =", p.gammas)

betas = [2.    0.125] , gammas = [0.125 0.   ]
betas = [2.5   0.125] , gammas = [0.125 0.   ]
betas = [2.    0.125] , gammas = [0.125 0.5  ]
betas = [2.5   0.125] , gammas = [0.125 0.5  ]


We can nest iterators arbitrarily, which allows for parameter sweeps and subsequent landscape analysis of any desired set of parameters. This functionality will be demoed in a separate notebook.

In [13]:
# Sweep over three parameters

iterator1 = QAOAParameterIterator(params,"betas[0]",np.arange(0,1,0.5))
for p1 in iterator1:
    iterator2 = QAOAParameterIterator(p1,"betas[1]",np.arange(0,1,0.5))
    for p2 in iterator2:
        iterator3 = QAOAParameterIterator(p2,"gammas[0]",np.arange(0,1,0.5))
        for j in iterator3:
            print("betas =", j.betas, ",", "gammas =", j.gammas)

betas = [0. 0.] , gammas = [0.  0.5]
betas = [0. 0.] , gammas = [0.5 0.5]
betas = [0.  0.5] , gammas = [0.  0.5]
betas = [0.  0.5] , gammas = [0.5 0.5]
betas = [0.5 0. ] , gammas = [0.  0.5]
betas = [0.5 0. ] , gammas = [0.5 0.5]
betas = [0.5 0.5] , gammas = [0.  0.5]
betas = [0.5 0.5] , gammas = [0.5 0.5]


<a id='footnotes'></a>

## Footnotes

1. There are other hyperparameters that we will not consider in this notebook. For example, here we have assumed that the mixer Hamiltonian is simply the sum of Pauli X operators on all qubits. However, one could clearly consider other types of mixers - see for example [Ref 3](#references). 

<a id='references'></a>

## References

1. E. Farhi et al, __[*A Quantum Approximate Optimization Algorithm*](https://arxiv.org/abs/1411.4028)__

2. L. Zhou et al, __[*Quantum Approximate Optimization Algorithm: Performance, Mechanism, and Implementation on Near-Term Devices*](https://arxiv.org/abs/1812.01041)__

3. Z. Wang et al, __[*XY-mixers: analytical and numerical results for QAOA*](https://arxiv.org/pdf/1904.09314.pdf)__